In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras,gc,time
from keras.layers import *
from keras.models import Sequential,Model

### Cargar datos

In [6]:
folder = "../proyecto/datasets/"
df_independencia_verano = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-verano.csv")
df_independencia_invierno = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-invierno.csv")

df_condes_verano = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-verano.csv")
df_condes_invierno = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-invierno.csv")

df_independencia_verano.head()

,registered_on,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
0,1997-11-01 01:00:00,2.7,63.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1997-11-01 02:00:00,2.6,54.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,1997-11-01 03:00:00,2.2,53.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,1997-11-01 04:00:00,2.5,65.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,1997-11-01 05:00:00,2.3,118.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [3]:
df_independencia_verano.tail()

,registered_on,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
75081,2018-03-31 19:00:00,NaN,40.0,9.0,NaN,NaN,NaN,NaN,164.584,43.0000,21.5583,1.245590,NaN,0.0,0.0,6.86500
75082,2018-03-31 20:00:00,NaN,33.5,9.0,NaN,NaN,NaN,NaN,169.048,50.0000,19.5583,1.169250,NaN,0.0,0.0,5.95750
75083,2018-03-31 21:00:00,NaN,24.5,11.0,NaN,NaN,NaN,NaN,207.717,54.5833,17.8083,0.966588,NaN,0.0,0.0,8.92751
75084,2018-03-31 22:00:00,NaN,27.5,10.0,NaN,NaN,NaN,NaN,189.509,58.4167,16.6667,0.816365,NaN,0.0,0.0,4.88500
75085,2018-03-31 23:00:00,NaN,29.0,10.0,NaN,NaN,NaN,NaN,231.942,61.4167,15.9083,1.116220,NaN,0.0,0.0,2.41000


In [7]:
## ultimo año como test
df_condes_verano_train = df_condes_verano[(df_condes_verano["registered_on"]<"2017")]
df_condes_verano_test = df_condes_verano[df_condes_verano["registered_on"]>="2017"]
df_condes_verano_train.tail()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69390,2016-12-31 14:00:00,NaN,0.553417,53.0,24.0,7.0400,3.2450,10.28500,NaN,NaN,41.8333,26.6333,NaN,NaN,NaN,NaN,72.5233
69391,2016-12-31 15:00:00,NaN,0.467584,22.0,21.0,4.4825,2.7500,7.23250,NaN,NaN,39.3333,26.3167,NaN,NaN,NaN,NaN,58.9725
69392,2016-12-31 16:00:00,NaN,0.459000,16.5,13.0,3.4925,2.9975,6.49000,NaN,NaN,37.3333,26.0583,NaN,NaN,NaN,NaN,50.3033
69393,2016-12-31 17:00:00,NaN,0.459000,15.0,14.0,3.9050,3.4100,7.31500,NaN,NaN,36.3333,25.1417,NaN,NaN,NaN,NaN,42.9808
69394,2016-12-31 18:00:00,NaN,0.450417,NaN,NaN,6.6275,3.1625,9.79001,NaN,NaN,38.8333,24.1750,NaN,NaN,NaN,NaN,NaN


In [14]:
df_condes_verano_test.head()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69395,2017-01-03 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,28.834697,NaN
69396,2017-01-03 14:00:00,NaN,NaN,19.5,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,24.715235,NaN
69397,2017-01-03 15:00:00,NaN,0.836667,13.5,18.0,10.2575,1.7600,12.0175,NaN,NaN,14.9167,28.8167,NaN,NaN,0.0,24.537322,73.6176
69398,2017-01-03 16:00:00,NaN,0.768001,19.5,2.0,10.8350,2.5025,13.3375,NaN,NaN,12.2500,28.2917,NaN,NaN,0.0,17.778006,57.6258
69399,2017-01-03 17:00:00,NaN,0.793750,72.0,10.0,11.5775,2.5850,14.1625,NaN,NaN,14.5000,27.4250,NaN,NaN,0.0,9.862870,51.9867


In [73]:
df_condes_verano.dropna(how='any')

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3


Si elimino todas las que tengan al menos un null se eliminan todos los registros

In [65]:
df_condes_verano_train.loc[:,rows_to_use].dropna(subset=["O3"])#(how="all",O3)

,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
0,1.100000,57.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.00000
1,0.900000,46.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.00000
2,0.799997,32.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.00000
3,0.700000,51.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.00000
4,0.599998,74.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00000
5,0.599998,115.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.00000
6,0.500000,86.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.00000
7,0.700000,89.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.00000
8,0.900000,142.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.00000
9,0.700000,110.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.00000


In [58]:
np.isnan(df_condes_verano_test.loc[:,rows_to_use].values)[:,-1]

array([ True,  True, False, ..., False, False, False])

### PRe PRocesarlos

In [8]:
#drop?

rows_to_use = ["CO","PM10","PM25","NO2","NO","NOX","SO2","WD","RH","TEMP","WS","HCNM","UVA","UVB","O3"]


In [99]:
## representacion

def create_sequences(dataframe,lag=1,horas=True):
    #preprocesamiento asumiendo dataset es una secuencia
    timestep = dataframe.loc[:,"registered_on"].values
    
    if not horas:
        indices_orden = []
        dia = timestep[0].split("-")[-1].split(" ")[0]
        aux_indices1 = 0
        aux_indices2 = 0 
        for dato in timestep[1:]:
            nuevo_dia = dato.split("-")[-1].split(" ")[0]
            aux_indices2+=1
            if dia != nuevo_dia:
                dia = nuevo_dia
                indices_orden.append([aux_indices1,aux_indices2])
                aux_indices1 = aux_indices2
        indices_orden.append([aux_indices1,aux_indices2+1])
        
        aux =  []
        data = dataframe.loc[:,rows_to_use].values
        for init,finit in indices_orden:
            aux+=[np.nanmax(data[init:finit],axis=0)]
        data = np.asarray(aux)
        #dropnan... de O3
        data = data[~np.isnan(data)[:,-1]]
    else:
        #dropnan... de o3
        data = dataframe.loc[:,rows_to_use].dropna(subset=["O3"]).values
    
    #rellenar nans de valores nada?
    
    dataX = []
    dataY =[]
    for t_plus_1 in range(lag,len(data)):
        #se crea el Y (target)
        predecir = data[t_plus_1][-1] #ultima columna es O3
        dataY.append(predecir)
        
        #se crea el X (inputs) columnas para predecir Y
        columnasX = []
        for i in np.arange(lag,0,-1): #para los valores anteriores al t_plus_1 durante un lag
            columnasX.append( data[t_plus_1-i] )
        dataX.append(columnasX)
    return np.array(dataX),np.array(dataY).reshape(len(dataY),1)

In [100]:
lag = 3
trainX, trainY = create_sequences(df_condes_verano_train, lag,horas=False)
testX, testY = create_sequences(df_condes_verano_test, lag,horas=False)
trainX.shape

/home/fmena/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: All-NaN slice encountered


(2931, 3, 15)

In [89]:
def trid_to_2d(array,backward=False):
    if backward:
        return array.reshape(array.shape[0]/lag, lag ,array.shape[-1])
    else:
        return array.reshape((np.prod(array.shape[:2]), array.shape[-1]))

In [146]:
trid_to_2d(trid_to_2d(trainX),backward=True).shape

(2932, 3, 16)

In [91]:
#calculo de estadisticos ("fit") MinMaxScaler
class MinMax(object):
    def __init__(self,aux=True):
        self.minimos = None
        self.maximos = None
        
    def fit(self,data):
        self.minimos = np.nanmin(data, axis=(0,1))#, keepdims=True)
        self.maximos = np.nanmax(data ,axis=(0,1))#, keepdims=True)
    
    def transform(self,data):
        #normalize between 0 and 1 every statisctic
        return (data-self.minimos) / (self.maximos-self.minimos)
    
    def inverse_transform(self,data):
        return data*(self.maximos-self.minimos) + self.minimos
    
scaler_model = MinMax()
scaler_model.fit(trainX)
X_train = scaler_model.transform(trainX)
X_test = scaler_model.transform(testX)
X_train.shape

(2931, 3, 15)

In [92]:
print("Minimos en train: ", np.nanmin(X_train, axis=(0,1)))
print("Minimos en test: ", np.nanmin(X_test, axis=(0,1)))

('Minimos en train: ', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))
('Minimos en test: ', array([6.89655125e-08, 2.74869110e-02, 2.99625468e-02, 1.12001868e-01,
       4.98162247e-03, 2.49952114e-02,            nan,            nan,
       2.50000000e-01, 8.71331507e-02,            nan,            nan,
       0.00000000e+00, 5.81265886e-04, 8.90017964e-02]))


/home/fmena/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: All-NaN slice encountered
  from ipykernel import kernelapp as app


In [93]:
print("Maximos en train: ", np.nanmax(X_train, axis=(0,1)))
print("Maximos en test: ", np.nanmax(X_test, axis=(0,1)))

('Maximos en train: ', array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))
('Maximos en test: ', array([0.48612073, 0.60536649, 0.52808989, 0.68545432, 0.24713179,
       0.39413212,        nan,        nan, 0.93349467, 0.85852949,
              nan,        nan, 0.02400363, 0.04777695, 0.99210778]))


/home/fmena/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: All-NaN slice encountered
  from ipykernel import kernelapp as app


In [114]:
#para el Y tambien...
from sklearn.preprocessing import MinMaxScaler
scaler_model = MinMaxScaler()
scaler_model.fit(trainY)
y_train = scaler_model.transform(trainY)
y_test = scaler_model.transform(testY)
y_train.shape

(2931, 1)

Características del modelo:
* Predecir el máximo de ozono $O_3$ del día siguiente (del período de 24 horas).
* Grano de predicción: Horas o día?
* Cuanta información anterior utilizar? ..3 dias
* Nulos? en $O_3$ o contaminantes?
    * Borrar
    * Mascara

In [ ]:
#predecir el máximo del dia siguiente...
#

## Modelo

In [120]:
batch_size = 64
epochs = 15
opt = 'adam'
loss_utilizada = 'mse' #?
nhidden = 64

In [136]:
from attention import *
modelo1 = Sequential()
#modelo1.add(LSTM(nhidden, input_shape=(timesteps, features), return_sequences=True))
modelo1.add(AttentionDecoder(nhidden, nhidden,input_shape=(timesteps,features),return_sequences=False)) #que son esos  (units, output_dim)

#modelo1.add(LSTM(nhidden, return_sequences=False))

modelo1.add(Dense(1,activation='linear')) #1 prediccion?
modelo1.compile(loss=loss_utilizada, optimizer=opt, metrics=['acc'])
modelo1.summary()
# train LSTM

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
AttentionDecoder (AttentionD (None, 3, 64)             43008     
_________________________________________________________________
dense_12 (Dense)             (None, 3, 1)              65        
Total params: 43,073
Trainable params: 43,073
Non-trainable params: 0
_________________________________________________________________


In [107]:
#https://github.com/farizrahman4u/seq2seq
import seq2seq
from seq2seq.models import SimpleSeq2Seq

model = SimpleSeq2Seq(input_dim=5, hidden_dim=10, output_length=8, output_dim=8)
model.compile(loss='mse', optimizer='rmsprop')

ImportError: No module named recurrentshop

In [161]:
valor_mascara = np.nan
timesteps = X_train.shape[1]
features = X_train.shape[2] #todo excepto o3

def attention_multiply(vects):
    encoder, attention = vects
    return K.batch_dot(attention,encoder, axes=1)

def modelo(tipo):
    #return sequence false si es a nivel de dia
    #return sequence true y GlobalMaxPooling1D() si es a nivel de hora
    model = Sequential()
    if tipo=="sin atencion":
        
        #model.add(Masking(mask_value=valor_mascara, input_shape=(timesteps, features)))

        model.add(LSTM(nhidden,input_shape=(timesteps, features),return_sequences=False))
        #model.add(LSTM(32,return_sequence=False))
        
    elif tipo=="atencion":
        model.add(LSTM(nhidden, input_shape=(timesteps, features), return_sequences=True))
        model.add(AttentionDecoder(nhidden, nhidden)) #que son esos  (units, output_dim)

        model.add(LSTM(nhidden, return_sequences=False))

    elif tipo=="atencion a mano":
        model.add(LSTM(nhidden, input_shape=(timesteps, features), return_sequences=True))
        
        attention = TimeDistributed(Dense(timesteps, activation='tanh'))(model.layers[-1].output)
        #softmax a las antenciones sobre todo T
        attention = Permute([2, 1])(attention)
        attention = Activation('softmax')(attention) 
        attention = Permute([2, 1])(attention)
        outp = Dot(axes=1)([attention,model.layers[-1].output])

        
        model = Model(inputs=model.inputs[0],outputs=outp)
        print(model.layers)
        #model.summary()
        
        model2 = Sequential(layers=model.layers)
                  
        #esta en la tarea...
    
    elif tipo=="atencion libreria":
        return #pendiente
    
    model.add(Dense(1,activation='linear')) #1 prediccion?

    #model.add(TimeDistributed(Dense(1,activation='linear'))) #varias prediccions--a nivel de horario..
    #model.add(GlobalMaxPooling1D())
    
    model.compile(loss=loss_utilizada,optimizer=opt)
    model.summary()
    return model

modelo1 = modelo("sin atencion")
modelo2 = modelo("atencion")
modelo3 = modelo("atencion a mano")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_101 (LSTM)              (None, 64)                20480     
_________________________________________________________________
dense_77 (Dense)             (None, 1)                 65        
Total params: 20,545
Trainable params: 20,545
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_102 (LSTM)              (None, 3, 64)             20480     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 3, 64)             61824     
_________________________________________________________________
lstm_103 (LSTM)              (None, 64)                33024     
_________________________________________________________________
dense_

ValueError: A merge layer should be called on a list of inputs.

In [127]:
modelo1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Train on 2931 samples, validate on 195 samples
Epoch 1/15
2931/2931 [==============================] - 17s 6ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/15
2931/2931 [==============================] - 1s 354us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/15
2931/2931 [==============================] - 1s 363us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/15
2931/2931 [==============================] - 1s 361us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/15
2931/2931 [==============================] - 1s 368us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/15
2931/2931 [==============================] - 1s 369us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/15
2931/2931 [==============================] - 1s 370us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc:

KeyboardInterrupt: 

In [9]:
## evaluación?
rmse?

#if es hora.. max
# si es por dia ya predice max